# Data Analysis for Android and iOS mobile apps

Our main goal of this project is to find profitable apps in Android and iOS market. Our data analysts intended to make data-driven decisions for companies for next stage of app development.

We build apps that are free to download and install. The revenue is influenced by the number of users of our application. This project's goal is to analyze data to help our developers understand what tupe of apps are likely to attract more users.

## Loading datasets to the project
The dataset used in this project: [Android App database](https://www.kaggle.com/lava18/google-play-store-apps), [IOS_App_database](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

In [1]:
opened_file = open('AppleStore.csv')
from csv import reader
read_file = reader(opened_file)
IOS = list(read_file)
IOS_header = IOS[0] 
IOS_data = IOS[1:]

opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
Android = list(read_file)
Android_header = Android[0]
Android_data = Android[1:]

Create a exploraton function for data inspection

In [2]:
def explore(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    
    if rows_and_columns:
        print('Number of rows', len(dataset))
        print('Number of columns', len(dataset[0]))

In [3]:
# print(Android_data[0])
# explore(IOS_data, 0, 4)
# explore(Android_data, 0, 4)

explore(IOS_data,0, 1, rows_and_columns=True)
print('\n')
explore(Android_data,0, 1, rows_and_columns=True)
print('\n')



['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows 7197
Number of columns 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows 10841
Number of columns 13




## Data Inspection
Let explore the headers of each dataset and try to identify the columns that could help with our analysis.

In [4]:
print('Android Header\n\n' + str(Android_header))
print('\n')
print('IOS Header\n\n' + str(IOS_header))

Android Header

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


IOS Header

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


We see that for Google Play dataset has 10841 apps and 13 columns. By inspection, the columns might be useful for the analysis are 'Apps', 'Rating', 'Reviews', 'Type', 'Price', 'Content Rating', 'Genres'.

Apply store data has 7197 apps and 16 columns. The columns might be useful are 'track_name', 'Price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'cont_rating', 'prime_genre'. 

# Data cleaning
Look for missing data

In [5]:
print(Android_data[10472])
del Android_data[10472]

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


### Part One: Check for duplicating dataset

In [6]:
duplicate_apps = []
unique_apps = []

for app in Android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps', len(duplicate_apps))
print('\n')
print('Example of duplicate apps', duplicate_apps[:15])

Number of duplicate apps 1181


Example of duplicate apps ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Here, we encounter duplicated data, one thing we could do is remov the duplicate row randomly, but there is another better way. Let's consider the app 'Instagram' as an example. 'Instagram' app contains duplicated data.

In [7]:
for app in Android_data:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


By inspection, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. This shows that the data was collected at different times. The higher the number of reviews, the more recent the data should be. So, we will keep the row with highest number of reviews and remove the other entries for any given app.

### Part Two: Building Dictionary

In [8]:
reviews_max = {}

for app in Android_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
 

The previous code, we found that there are 1,181 cases for duplicated data. So, the length of our dictionary should be 1,181. Next, we have to remove duplicated cases from our dataset.

Criterions: For any given apps in Android store, we only consider the data  for later analysis based on the highest number of available reviews.

In [9]:
android_clean = []
already_added = []

for app in Android_data:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)
        

In [10]:
explore(android_clean,0,1, rows_and_columns=True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows 9659
Number of columns 13


The length of Android app data after cleaning should has a length of 9659 

## Remove Non-English apps

Our company intended to create applications that are directed toward an English-speaking audience. Here, we filter out the non-english apps. A function is created to check whether the app is designed in english or not. 

In [11]:
def detectenglish(name):
    count = 0;
    for charactor in name:
        if ord(charactor) > 127:
            count += 1
        if count > 3:
            return False
    return True

This function checks the name of each application. All English apps with up to three special characters will still be labeled as English. To check whether the function is working or not, we tests our functions with several app names.


In [12]:
name_app1 = '爱奇艺PPS -《欢乐颂2》电视剧热播'
name_app2 = 'Docs To Go™ Free Office Suite'
name_app3 = 'Instachat 😜'

print(detectenglish(name_app1))
print(detectenglish(name_app2))
print(detectenglish(name_app3))

False
True
True


Apply the filtering function to the clean Android dataset and the apple store dataset.

In [13]:
android_clean_noneng = []
IOS_data_noneng = []

for app in android_clean:
    name = app[0]
    if detectenglish(name):
        android_clean_noneng.append(app)

for app in IOS_data:
    name = app[1] #Data name has index 1
    if detectenglish(name):
        IOS_data_noneng.append(app)


In [14]:
explore(android_clean,0, 1,  rows_and_columns=True)
explore(android_clean_noneng,0, 1,  rows_and_columns=True)
explore(IOS_data,0, 1,  rows_and_columns=True)
explore(IOS_data_noneng,0, 1,  rows_and_columns=True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows 9659
Number of columns 13
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows 9614
Number of columns 13
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows 7197
Number of columns 16
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows 6183
Number of columns 16


We can see that there are 9614 android apps and 6183 iOS apps available in our dataset.

## Removing paid applications in the market
Our company only build apps that are free to download and install, and the main source of revenue consists of in-app ads. So, we have to isolate only free apps for our analysis

In [15]:
app = android_clean_noneng[0]
android_clean_noneng_free = []
IOS_clean_noneng_free = []

for app in android_clean_noneng:
    Price = app[7]
    if Price == '0':
        android_clean_noneng_free.append(app)
        
for app in IOS_data_noneng:
    Price = app[4]
    if (Price == '0.0'):
        IOS_clean_noneng_free.append(app)
        
print(len(android_clean_noneng_free))
print(len(IOS_clean_noneng_free))

Android_final = android_clean_noneng_free
IOS_final = IOS_clean_noneng_free

8864
3222


So, there are 8864 androids apps and 3222 iOS apps available in our dataset.

# Most Common Apps by Genre

To simplify our analysis, we build 2 functions to generate the frequency table. 

In [16]:
def freq_table(data_set, index):
    table = {}
    table_percentage = {}
    total = 0
    for row in data_set:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1       
    for row in table:
        row_fraction = float(table[row]) / total
        table_percentage[row] = row_fraction * 100
    return table_percentage

def diplay_table(data_set, index):
    
    table = freq_table(data_set, index)
    table_display  = []
    
    for row in table:
        key_val_as_tuple = (table[row] ,row) # percentage is a tublet
        table_display.append(key_val_as_tuple) # List is appended by tublet
    
    #Sort can be used on Dictionary
    table_sorted = sorted(table_display, reverse = True)
    
    for input_ in table_sorted:
        print(input_[1], ':', input_[0])


We start by examining the frequency table for the prime_genre column of the App Store data set.


In [17]:
diplay_table(IOS_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In the IOS market, we can inspect that more than half (58.16%) of mobile apps are games. Entertainment apps are close to 8%. The general impression of the IOS app store (english, free) is dominated by the apps that are related to game, entainment, photo and video. 

Apps designed for practical purposes (eduction, shopping, utilities, productivity and lifestyle are less numerous than entertainment apps. 'Prime genre' only study how numerous of app kinds behaves but it does not necessary reflect the number of users. 

Then, we continue by examining the frequency table for the Category column of the Android Store data set.


In [18]:
diplay_table(Android_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

There are not many apps designed for fun. Most of the apps are targeting family and there are more practical apps in Android market. (family, tools, business, productivity, finace, medical and etc.)

In [19]:
Android_sorted = diplay_table(Android_final, -4)
print(Android_sorted)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Genres and category columns are quite similar. We can observe that the genre column is more granular. We are going to focus more on the big picture for the Android market and let's use the Category column for the analysis.

# Which genre are the most popular?

In [20]:
IOS_sorted = freq_table(IOS_final, -5)

In [21]:
for genre in IOS_sorted:
    total = 0
    len_genre = 0
    for app in IOS_final:
        genre_app = app[-5]
        if genre_app == genre:
            total +=  float(app[5])
            len_genre += 1
    avg_user_rating = total / len_genre
    print(genre, avg_user_rating)

Social Networking 71548.34905660378
Photo & Video 28441.54375
Games 22788.6696905016
Music 57326.530303030304
Reference 74942.11111111111
Health & Fitness 23298.015384615384
Weather 52279.892857142855
Utilities 18684.456790123455
Travel 28243.8
Shopping 26919.690476190477
News 21248.023255813954
Navigation 86090.33333333333
Lifestyle 16485.764705882353
Entertainment 14029.830708661417
Food & Drink 33333.92307692308
Sports 23008.898550724636
Book 39758.5
Finance 31467.944444444445
Education 7003.983050847458
Productivity 21028.410714285714
Business 7491.117647058823
Catalogs 4004.0
Medical 612.0


Apps related to 'Social Networking', 'Reference', 'Weather', 'Navigation' and 'Book' have the best average user rating. 

In [22]:
for app in IOS_final:
    genre = app[-5]
    if genre == 'Social Networking':
        print(app[1], ':', app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

In [23]:
for app in IOS_final:
    genre = app[-5]
    if genre == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


In [24]:
for app in IOS_final:
    genre = app[-5]
    if genre == 'Weather':
        print(app[1], ':', app[5])

The Weather Channel: Forecast, Radar & Alerts : 495626
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking : 208648
WeatherBug - Local Weather, Radar, Maps, Alerts : 188583
MyRadar NOAA Weather Radar Forecast : 150158
AccuWeather - Weather for Life : 144214
Yahoo Weather : 112603
Weather Underground: Custom Forecast & Local Radar : 49192
NOAA Weather Radar - Weather Forecast & HD Radar : 45696
Weather Live Free - Weather Forecast & Alerts : 35702
Storm Radar : 22792
QuakeFeed Earthquake Map, Alerts, and News : 6081
Moji Weather - Free Weather Forecast : 2333
Hurricane by American Red Cross : 1158
Forecast Bar : 375
Hurricane Tracker WESH 2 Orlando, Central Florida : 203
FEMA : 128
iWeather - World weather forecast : 80
Weather - Radar - Storm with Morecast App : 78
Yurekuru Call : 53
Weather & Radar : 37
WRAL Weather Alert : 25
Météo-France : 24
JaxReady : 22
Freddy the Frogcaster's Weather Station : 14
Almanac Long-Range Weather Forecast : 12
TodayAir

In [25]:
for app in IOS_final:
    genre = app[-5]
    if genre == 'Navigation':
        print(app[1], ':', app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


In [26]:
for app in IOS_final:
    genre = app[-5]
    if genre == 'Book':
        print(app[1], ':', app[5])

Kindle – Read eBooks, Magazines & Textbooks : 252076
Audible – audio books, original series & podcasts : 105274
Color Therapy Adult Coloring Book for Adults : 84062
OverDrive – Library eBooks and Audiobooks : 65450
HOOKED - Chat Stories : 47829
BookShout: Read eBooks & Track Your Reading Goals : 879
Dr. Seuss Treasury — 50 best kids books : 451
Green Riding Hood : 392
Weirdwood Manor : 197
MangaZERO - comic reader : 9
ikouhoushi : 0
MangaTiara - love comic reader : 0
謎解き : 0
謎解き2016 : 0


In [27]:
diplay_table(Android_final,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


In [28]:
Android_Category = freq_table(Android_final, 1)

for category in Android_Category:
    
    genre = category
    total = 0
    len_install = 0
    
    for app in Android_final:
        app_genre = app[1]
        
        if(genre == app_genre):
            install_number = app[5]
            install_number = install_number.replace('+','')
            install_number = install_number.replace(',','')
            
            total += int(install_number)
            len_install += 1
    
    average_num_install = total / len_install
    print(genre, ':',average_num_install)



ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

In [29]:
for app in Android_final:
    if(app[1]=='COMMUNICATION'):
        print(app[0], ':',app[5])

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
TracFone My Account : 1,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
My magenta : 1,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Seznam.cz : 1,000,000+
Antillean Gold Telegram (original version) : 100,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Microsoft Edge : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Calls & Text by Mo+ : 5,000,000+
free 